In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
def load_model_tokenizer():
    peft_model_id = "lora-flan-t5-large-sentiment"
    config = PeftConfig.from_pretrained(peft_model_id)

    model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

    model = PeftModel.from_pretrained(model, peft_model_id).to("cpu")
    model.eval()
    return model, tokenizer

In [3]:
model, tokenizer = load_model_tokenizer()

In [4]:
def inference(model, tokenizer, input_sent):
    input_ids = tokenizer(input_sent, return_tensors="pt", truncation=True, max_length=256).input_ids.to("cpu")
    outputs = model.generate(input_ids=input_ids, top_p=0.9, max_length=256)
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [42]:
prompt = """
        Human: I absolutely love my SR Watch! The design is sleek, the battery life is impressive, and it syncs seamlessly with my smartphone. It has truly enhanced my daily routine.
        Assistant: Positive

        Human: My experience with the SR Watch has been disappointing. The battery drains quickly, and the interface is clunky. I expected better performance considering the price.
        Assistant: Negative

        Human: The SR Watch has some good features, but there are also some drawbacks. The design is modern, but the battery life could be better. Overall, it's an okay product.
        Assistant: Neutral
    """

In [43]:
input_review = input("Enter your review: ")

Enter your review:  The SR Watch has some good features, but there are also some drawbacks. The design is modern, but the battery life could be better. Overall, it's an okay product.


In [44]:
input_review = prompt + "Human: " + input_review + ".\n Assistant:"

In [45]:
input_review

"\n        Human: I absolutely love my SR Watch! The design is sleek, the battery life is impressive, and it syncs seamlessly with my smartphone. It has truly enhanced my daily routine.\n        Assistant: Positive\n\n        Human: My experience with the SR Watch has been disappointing. The battery drains quickly, and the interface is clunky. I expected better performance considering the price.\n        Assistant: Negative\n\n        Human: The SR Watch has some good features, but there are also some drawbacks. The design is modern, but the battery life could be better. Overall, it's an okay product.\n        Assistant: Neutral\n    Human: The SR Watch has some good features, but there are also some drawbacks. The design is modern, but the battery life could be better. Overall, it's an okay product..\n Assistant:"

In [46]:
inference(model, tokenizer, input_review)

'Positive'